# Vishal Bhalla | 10/30/2016

## Gradient Boosting Model

Below requires the output from the feature extraction notebook, 4_GeoEmpTitle_Feature_Extraction.

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.04, loss='deviance', max_depth=2,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=200,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=2000, presort='auto', random_state=401,
              subsample=1.0, verbose=0, warm_start=False)


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.model_selection import train_test_split
from time import time
from sklearn.metrics import f1_score
from sklearn.preprocessing import scale

In [2]:
random_state=401
n_jobs = 1

In [3]:
cd Data

C:\Users\Vishal\Dropbox\Coursework\DS\KLC\Necessary\Model Generation 2014 Scripts\Data


In [4]:
#Import output from other notebooks
df_reduce = pd.read_pickle('join_fw_zip_out_117.p')


In [5]:
df_reduce.shape

(232487, 38)

In [6]:
#df_reduce = dfc.copy()
#df_reduce = dfc.copy()
#del dfc

In [7]:
grades = {
'A1': 1,
'A2': 2,
'A3': 3,
'A4': 4,
'A5': 5,
'B1': 6,
'B2': 7,
'B3': 8,
'B4': 9,
'B5': 10,
'C1': 11,
'C2': 12,
'C3': 13,
'C4': 14,
'C5': 15,
'D1': 16,
'D2': 17,
'D3': 18,
'D4': 19,
'D5': 20,
'E1': 21,
'E2': 22,
'E3': 23,
'E4': 24,
'E5': 25,
'F1': 26,
'F2': 27,
'F3': 28,
'F4': 29,
'F5': 30,
'G1': 31,
'G2': 32,
'G3': 33,
'G4': 34,
'G5': 35
}

In [8]:
#df_reduce['grade_val'] = df_reduce.sub_grade.map(grades)
df_reduce['emp_length_cat'] = df_reduce.emp_length_cat.astype('category')

In [9]:
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import scale
from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split

In [10]:
def my_scorer(clf, X, y):
    y_pred_prob = clf.predict_proba(X)[:,1]
    return roc_auc_score(y, y_pred_prob)

In [11]:
sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=random_state)
scoring_function = my_scorer

In [12]:
features = [u'loan_amnt', 'term', 'int_rate',  
            u'home_ownership', u'annual_inc',u'verification_status', u'purpose', 
            u'dti', u'delinq_2yrs', u'earliest_cr_line', u'inq_last_6mths', u'open_acc', u'pub_rec', u'revol_bal', 
            u'revol_util', u'initial_list_status', u'collections_12_mths_ex_med', u'mort_acc',#u'emp_title_flag',  
            u'fico_avg', 'emp_length_cat', #u'fw_emp_title_gt35','fw_emp_title_gt2',
            'fw_emp_title_gt27', 
            u'Unemployed_Rate', u'Child_Rate', 
            u'RE_Tax_Rate', 
            #u'jw_emp_match', 
            #u'jw_gt_25', u'grade_val',
            'sub_grade'
           ]

target = ['loan_label']

X=df_reduce[features]
print X.columns
X = pd.get_dummies(X)
col = X.columns
print col
X = scale(X)
print X.shape
y=df_reduce[target].values.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y,  stratify=y, test_size=.3, random_state=random_state)

Index([u'loan_amnt', u'term', u'int_rate', u'home_ownership', u'annual_inc', u'verification_status', u'purpose', u'dti', u'delinq_2yrs', u'earliest_cr_line', u'inq_last_6mths', u'open_acc', u'pub_rec', u'revol_bal', u'revol_util', u'initial_list_status', u'collections_12_mths_ex_med', u'mort_acc', u'fico_avg', u'emp_length_cat', u'fw_emp_title_gt27', u'Unemployed_Rate', u'Child_Rate', u'RE_Tax_Rate', u'sub_grade'], dtype='object')
Index([u'loan_amnt', u'int_rate', u'annual_inc', u'dti', u'delinq_2yrs', u'earliest_cr_line', u'inq_last_6mths', u'open_acc', u'pub_rec', u'revol_bal',
       ...
       u'sub_grade_F1', u'sub_grade_F2', u'sub_grade_F3', u'sub_grade_F4', u'sub_grade_F5', u'sub_grade_G1', u'sub_grade_G2', u'sub_grade_G3', u'sub_grade_G4', u'sub_grade_G5'], dtype='object', length=243)
(232487L, 243L)


### Parameter Tuning
1. min_samples_split = 500,1000 : This should be ~0.5-1% of total values. Since this is imbalanced class problem, we’ll take a small value from the range.
2. min_samples_leaf = 50 : Can be selected based on intuition. This is just used for preventing overfitting and again a small value because of imbalanced classes.
3. max_depth = 8 : Should be chosen (5-8) based on the number of observations and predictors. This has 87K rows and 49 columns so lets take 8 here.
4. max_features = ‘sqrt’ : Its a general thumb-rule to start with square root.

Steps, run for min sampels and max depth.  Then also see if you can replicate with lower ntrees to 100 to see if you get the same outputs as compared to 1000 trees.  Max depth was the most influential with 2 beating out 5.  lower learn rate did better as well.

1.Num trees/learning rate for speed
2.max_depth and num_samples_split
3.min_samples per leaf (see if min_samples or num_samples_split is more sen)
4.tune max features


In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV 

In [20]:
range_ntrees = np.arange(150,1150,50)

scores = []
for n in range_ntrees:
    start = time()  
    gbm = GradientBoostingClassifier(random_state=random_state, n_estimators=n, learning_rate=.04, min_samples_leaf=1100)
    gbm.fit(X_train,y_train)
    y_pred_prob1 = gbm.predict_proba(X_test)[:,1]
    score = roc_auc_score(y_test, y_pred_prob1)
    end = time()
    duration = (end - start)/60
    print n, score, duration
    scores.append(dict(n_iterations=n, auc=score, duration=duration))
    
scores = pd.DataFrame(scores)
print scores


150 0.692231613275 10.0228333354
200 0.693725141834 13.8000333349
250 0.694590432529 16.6191499988
300 0.69508098771 20.1582333366
350 0.695425878675 23.3032333334
400 0.69562276498 26.883100001
450 0.695739329956 30.1652999997
500 0.695816722801 32.7237999996
550 0.69590184614 35.8766499996
600 0.695937808465 38.5164000034
650 0.695943786557 41.9030833324
700 0.696000416278 44.857766668
750 0.695993342203 48.1392666658
800 0.695958546192 51.1695500016
850 0.695966997573 54.2514500022
900 0.696046392884 58.8308166663
950 0.696020989901 63.089716665
1000 0.696044638529 62.5576666673
1050 0.6959772795 66.031766665
1100 0.695934553726 67.3217333317
         auc   duration  n_iterations
0   0.692232  10.022833           150
1   0.693725  13.800033           200
2   0.694590  16.619150           250
3   0.695081  20.158233           300
4   0.695426  23.303233           350
5   0.695623  26.883100           400
6   0.695739  30.165300           450
7   0.695817  32.723800           500
8   

## HERE!!!! After you Select Optimum Amount of Trees, then run Next one

In [14]:
range_depth = [3,5,8] #greater than 9 if it performs better
range_min_samples_leaf = [1100, 1600, 2200] #1100
range_learn_rate = [.04, .08]

parameters=dict(learning_rate=range_learn_rate, max_depth=range_depth, 
                min_samples_leaf=range_min_samples_leaf)

gbm = GradientBoostingClassifier(random_state=random_state, n_estimators=700)
grid_rand_gbm = RandomizedSearchCV(estimator=gbm, param_distributions=parameters, cv=sss, n_jobs=n_jobs, scoring=my_scorer, refit=True)


#import caffeine
start = time()
grid_rand_gbm.fit(X_train,y_train)
end = time()
print "{:.4f} seconds".format(end - start)
print grid_rand_gbm.best_score_
print grid_rand_gbm.best_estimator_
y_pred_prob1 = grid_rand_gbm.predict_proba(X_test)[:,1]
score = roc_auc_score(y_test, y_pred_prob1)
print "Test Score",score
grid_rand_gbm.cv_results_
#caffeine.off()

115047.8940 seconds
0.692483702402
GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.04, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=2200,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=700, presort='auto', random_state=401,
              subsample=1.0, verbose=0, warm_start=False)
Test Score 0.695898034618


{'mean_fit_time': array([ 5247.14200004,  2012.39199996,  1998.00499988,  5403.66766667,
         3624.73600006,  3498.89366674,  5986.85966659,  5677.12266668,
         1949.82833338,  1964.93799996]),
 'mean_score_time': array([ 3.11333323,  1.77566663,  1.78266676,  3.15266665,  2.33533335,
         2.32999992,  3.20166667,  3.19933335,  1.73433336,  1.71466668]),
 'mean_test_score': array([ 0.68638291,  0.69238136,  0.6924837 ,  0.6849404 ,  0.69156642,
         0.69195346,  0.68930824,  0.6899715 ,  0.69130568,  0.69146559]),
 'mean_train_score': array([ 0.77920873,  0.71450835,  0.71329014,  0.79087174,  0.7364272 ,
         0.73150693,  0.7759416 ,  0.76349714,  0.72229293,  0.72441504]),
 'param_learning_rate': masked_array(data = [0.08 0.04 0.04 0.08 0.04 0.04 0.04 0.04 0.08 0.08],
              mask = [False False False False False False False False False False],
        fill_value = ?),
 'param_max_depth': masked_array(data = [8 3 3 8 5 5 8 8 3 3],
              mask = [Fals

## Haven't done anything here

In [19]:
#range_ntrees = 500 #[500,2000] leave at 500 for now 
range_depth = [2,3,7,9] #greater than 9 if it performs better
range_min_samples_leaf = [500,1100,2200] #1100
range_learn_rate = [.02, .04, .08] #0.01-0.05


parameters=dict(learning_rate=range_learn_rate, max_depth=range_depth, 
                min_samples_leaf=range_min_samples_leaf)

gbm = GradientBoostingClassifier(random_state=random_state, n_estimators=500)
grid_rand_gbm = RandomizedSearchCV(estimator=gbm, param_distributions=parameters, cv=sss, n_jobs=1, scoring=my_scorer, refit=True)

#import caffeine
start = time()
grid_rand_gbm.fit(X_train,y_train)
end = time()
print "{:.4f} seconds".format(end - start)
print grid_rand_gbm.best_score_
print grid_rand_gbm.best_estimator_
y_pred_prob1 = grid_rand_gbm.predict_proba(X_test)[:,1]
score = roc_auc_score(y_test, y_pred_prob1)
print "Test Score",score
grid_rand_gbm.cv_results_
#caffeine.off()

{'mean_fit_time': array([ 3466.15633329,   853.921     ,  4037.03700002,  1398.16399995,
         3791.23133334,  5057.92233332,  4508.09033322,   846.14733338,
         1370.74533335,   859.42100008]),
 'mean_score_time': array([ 2.21833324,  0.93966667,  2.31900001,  1.29233336,  2.06133334,
         2.35633334,  2.26266678,  0.93566672,  1.23866661,  0.96900002]),
 'mean_test_score': array([ 0.6922072 ,  0.69229065,  0.68762318,  0.69297316,  0.6873265 ,
         0.6836477 ,  0.6913673 ,  0.69285772,  0.69257861,  0.69172028]),
 'mean_train_score': array([ 0.73953131,  0.70658174,  0.77225418,  0.71041583,  0.80093606,
         0.84031426,  0.74962506,  0.70665486,  0.71954979,  0.70138868]),
 'param_learning_rate': masked_array(data = [0.04 0.08 0.08 0.04 0.08 0.08 0.02 0.08 0.08 0.04],
              mask = [False False False False False False False False False False],
        fill_value = ?),
 'param_max_depth': masked_array(data = [7 2 9 3 7 9 7 2 3 2],
              mask = [Fals